In [36]:
import os
import sys
import numpy as np
from collections import defaultdict, Counter
import time
from matplotlib import pyplot
import json5
from matplotlib import pyplot as pyplot
from scipy.cluster.hierarchy import dendrogram
import pandas

import importlib
import duhportinf
import duhportinf._grouper
import duhportinf._optimize
import duhportinf._vectorizer
import duhportinf.busdef
import duhportinf.main 
from   duhportinf.busdef import BusDef
from   duhportinf._optimize import MatchCost
from   duhportinf import util

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
from collections import deque
import copy
import json

def update(d, u):
    for k, v in u.items():
        if isinstance(v, collections.Mapping):
            # special case when string is already inserted
            if k in d and isinstance(d[k], str):
                d[k] = {'_':d[k]}
            d[k] = update(d.get(k, {}), v)
        else:
            d[k] = v
    return d

class Bundle(object):
    
    @property
    def tree(self):
        return copy.deepcopy(self._tree)
    
    def __init__(self, names):
        self._tree = {}
        for name in names:
            update(self._tree, self.subtree_from_name(name))
        self._format_vectors()
        
    def subtree_from_name(self, name):
        """
        convert name to subtree to be inserted
        """
        words = util.words_from_name(name)
        # leaf should map to port name
        tree_dict = {words[-1] : name}
        for w in reversed(words[:-1]):
            tree_dict = {w: tree_dict}
        return tree_dict
    
    def _format_vectors(self):
        """
        format vectors of the tree in place
        """
        stack = deque([(None, None, self._tree)])
        while len(stack) > 0:
            parent, pkey, curr = stack.popleft()
            if self.is_vector(curr):
                vector = [p for i, p in 
                    sorted([(idx, self.deref_singleton_path(stree)) for idx, stree in curr.items()])
                ]
                parent[pkey] = vector
            else:
                for k, child in curr.items():
                    if not self.is_leaf(child):
                        stack.appendleft((curr, k, child))

    def is_vector(self, stree):
        return (
            not self.is_leaf(stree) and
            all([self.is_singleton_path(ss) for ss in stree.values()]) and
            self.is_range(stree.keys())
        )
       
    def is_range(self, keys):
        if not all([k.isdigit() for k in keys]):
            return False
        idx = [int(k) for k in keys]
        return util.is_range(idx)
    
    def is_leaf(self, stree):
        # vector or string
        return type(stree) in [list, str]
    
    def deref_singleton_path(self, stree):
        if self.is_leaf(stree):
            return stree
        else:
            assert len(stree) == 1
            sstree = next(iter(stree.values()))
            return self.deref_singleton_path(sstree)
        
    def is_singleton_path(self, stree):
        if self.is_leaf(stree):
            return True
        elif len(stree) > 1:
            return False
        else:
            sstree = next(iter(stree.values()))
            return self.is_singleton_path(sstree)
        

In [192]:
names = [
    'foo',
    'foo_bar1',
    'foo_bar2',
    'foo_baz1_n',
    'foo_baz2_n',
] 
b = Bundle(names)
print(json.dumps(b.tree, indent=2))

{
  "foo": {
    "_": "foo",
    "bar": [
      "foo_bar1",
      "foo_bar2"
    ],
    "baz": [
      "foo_baz1_n",
      "foo_baz2_n"
    ]
  }
}


In [37]:
block_root = '/ip-block-designs'
bus_spec_rootdir = '/bus-defs/specs'

In [38]:
def get_ports_from_json5(path):
    with open(path) as fin:
        block = json5.load(fin)
    return util.format_ports(block['definitions']['ports'])

In [39]:
pcie_json_path = os.path.join(
    block_root,
    'block-plda-pcie-controller',
    'PLDA_ws.json5',
)
ddr_json_path = os.path.join(
    block_root,
    'block-cadence-ddr3',
    'ddr.json5',
)
nvdla_json_path = os.path.join(
    block_root,
    'block-nvdla',
    'nvdla.json5',
)
pcie_ports  = get_ports_from_json5(pcie_json_path)
ddr_ports   = get_ports_from_json5(ddr_json_path)
nvdla_ports = get_ports_from_json5(nvdla_json_path)

In [40]:
bus_defs = duhportinf.load_bus_defs(bus_spec_rootdir)

loading 28 bus specs
  - done, loaded 50 bus defs with 535 required and 623 optional ports 


In [41]:
tmp_json_path = os.path.join(block_root, 'tmp', 'tmp.json5')
tmp_ports = get_ports_from_json5(tmp_json_path)

In [42]:
pg, Z, wn = duhportinf.get_port_grouper(ddr_ports)

In [44]:
dport = ('dfi_phy_wrlvl_cs0_n', 2, 1)
#print('vector root ids', len(pg.vector_root_ids))
#for nid in pg.vector_root_ids:
#    vports = list(sorted(pg.vector_nid_ports_map[nid]))
#    print('{} vector ports'.format(len(vports)), len(vports))
for _, interface in pg.get_initial_interfaces():
    if dport in set(interface.ports):
        print('interface width:{}, structed_width:{}'.format(
            interface.size,
            interface.structed_width,
        ))
        print(' - num vector bundles', len(list(interface.vector_bundles)))
        for b in interface.vector_bundles:
            print('    - prefix:{}, size:{}'.format(b.prefix, b.size))

interface width:146, structed_width:8
 - num vector bundles 8
    - prefix:dfi_lvl_pattern_, size:9
    - prefix:dfi_phy_wrlvl_cs, size:9
    - prefix:dfi_phy_rdlvl_gate_cs, size:9
    - prefix:dfi_phy_rdlvl_cs, size:9
    - prefix:dfi_rddata_cs_n_p1_, size:9
    - prefix:dfi_rddata_cs_n_, size:9
    - prefix:dfi_wrdata_cs_n_p1_, size:9
    - prefix:dfi_wrdata_cs_n_, size:9
interface width:168, structed_width:8
 - num vector bundles 8
    - prefix:dfi_lvl_pattern_, size:9
    - prefix:dfi_phy_wrlvl_cs, size:9
    - prefix:dfi_phy_rdlvl_gate_cs, size:9
    - prefix:dfi_phy_rdlvl_cs, size:9
    - prefix:dfi_rddata_cs_n_p1_, size:9
    - prefix:dfi_rddata_cs_n_, size:9
    - prefix:dfi_wrdata_cs_n_p1_, size:9
    - prefix:dfi_wrdata_cs_n_, size:9
interface width:136, structed_width:8
 - num vector bundles 8
    - prefix:dfi_lvl_pattern_, size:9
    - prefix:dfi_phy_wrlvl_cs, size:9
    - prefix:dfi_phy_rdlvl_gate_cs, size:9
    - prefix:dfi_phy_rdlvl_cs, size:9
    - prefix:dfi_rddata_cs_

In [583]:
# check if splitting port names on digits screws up initial port groups
pg, Z, wn = duhportinf.get_port_grouper(ddr_ports)

In [ ]:
df = pandas.DataFrame(Z)
df = df.astype({0:int, 1:int, 3:int})
df.to_csv('linkage-tree.csv', header=False, index=False)
df = pandas.DataFrame(wire_names)
df.to_csv('wire-names.csv', header=False, index=False)
df.head()

In [ ]:
def debug(ports, dport, bus_abstract_names):
    # filter to get debug bus defs
    bus_abstract_names = set(bus_abstract_names)
    dbus_defs = list(filter(
        lambda bd: bd.abstract_type.name in bus_abstract_names,
        bus_defs,
    ))
    
    pg, Z, wire_names = duhportinf._grouper.get_port_grouper(ports)
    pg_bus_pairings = duhportinf.main._get_bus_pairings(pg, dbus_defs)
    # filter only for port_group with dport
    pg_bus_pairings = list(filter(
        lambda x: dport in x[2],
        pg_bus_pairings,
    ))
    pg_bus_mappings = duhportinf.main._get_initial_bus_matches(pg, pg_bus_pairings)
    return list(map(lambda x: x[2:], pg_bus_mappings))
    

In [ ]:
port_path = '/ip-block-designs/tmp/tmp.json5' 
ports = duhportinf.get_ports_from_json5(port_path)
dport = ('front_port_axi4_0_ar_ready', 1, -1)
pg_bus_mappings = debug(ports, dport, ['AXI4_rtl'])

In [ ]:
stime = time.time()

nvdla_pg_bus_mappings = duhportinf.get_bus_matches(nvdla_ports, list(bus_defs))
etime = time.time()
print('total time: {}s'.format(etime-stime))

In [ ]:
util.dump_json_bus_candidates('testy.json', nvdla_json_path, nvdla_pg_bus_mappings)

In [ ]:
stime = time.time()

ddr_pg_bus_mappings = duhportinf.get_bus_matches(ddr_ports, list(bus_defs))

etime = time.time()
print('total time: {}s'.format(etime-stime))

In [46]:
s_pcie_ports = list(filter(lambda p: 'axi4' not in p[0], pcie_ports))
stime = time.time()
pcie_pg_bus_mappings = duhportinf.get_bus_matches(s_pcie_ports, list(bus_defs))

etime = time.time()
print('total time: {}s'.format(etime-stime))

hierarchically clustering ports and selecting port groups
  - done
initial bus pairing with port groups
  - done
bus mapping
  - done                                                                         
total time: 13.784493684768677s


In [51]:
dport = ('pl_pll_ack', 1, 1)
for interface, bus_mappings in pcie_pg_bus_mappings:
    if dport in interface.ports:
        print('port group')
        for port in interface.ports:
            print('  - ', port)
        print('bus mappings')
        for bm in bus_mappings:
            print('  - cost:{} bus_def:{}'.format(
                bm.cost,
                str(bm.bus_def.abstract_type),
            ))

port group
  -  ('pl_rxdata', None, 1)
  -  ('pl_rxstatus', None, 1)
  -  ('pl_phystatus', None, 1)
  -  ('pl_rxstandby', None, -1)
  -  ('pl_rxpolarity', None, -1)
  -  ('pl_txcompliance', None, -1)
  -  ('pl_txelecidle', None, -1)
  -  ('pl_txsyncheader', None, -1)
  -  ('pl_txstartblock', None, -1)
  -  ('pl_txdatavalid', None, -1)
  -  ('pl_txdatak', None, -1)
  -  ('pl_txdata', None, -1)
  -  ('pl_txdetectrx', None, -1)
  -  ('pl_pll_rate', 3, -1)
  -  ('pl_pll_ack', 1, 1)
  -  ('pl_width', 2, -1)
  -  ('pl_ltssm', 5, -1)
  -  ('pl_blockaligncontrol', 1, -1)
  -  ('pl_txdeemph', 1, -1)
  -  ('pl_txswing', 1, -1)
  -  ('pl_txmargin', 3, -1)
  -  ('pl_rate', 2, -1)
  -  ('pl_powerdown', 2, -1)
  -  ('pl_spor', 1, 1)
  -  ('pl_npor', 1, 1)
  -  ('pl_srst', 1, 1)
  -  ('pl_rstnp', 1, 1)
  -  ('pl_rstn', 1, 1)
  -  ('pl_rstn_srst_out', 1, -1)
  -  ('pl_pclk_change_ok', None, 1)
  -  ('pl_pclk_change_ack', None, -1)
  -  ('pl_pclk', 1, 1)
  -  ('pl_pclk_rate', 3, -1)
bus mappings
  - co

In [ ]:
def dump_nonaxi_dendogram(path, ports):
    assert path.endswith('.pdf')
    nonaxi_ports = list(filter(
        lambda p: not p[0].startswith('axi'),
        ports,
    ))
    _, Z, wire_names = abxportinf.get_port_grouper(nonaxi_ports)
    fig, ax = pyplot.subplots(1,1, figsize=(10,40))
    _ = dendrogram(
        Z,
        ax=ax,
        orientation='left',
        labels=wire_names,
    )
    pyplot.savefig(path)


In [ ]:
dump_nonaxi_dendogram('ddr-nonaxi-dendogram.pdf', ddr_ports)
dump_nonaxi_dendogram('pcie-nonaxi-dendogram.pdf', pcie_ports)

In [ ]:
util.dump_json_bus_candidates('ddr-busprop.json', ddr_pg_bus_mappings)
util.dump_json_bus_candidates('pcie-busprop.json', pcie_pg_bus_mappings)